In [1]:
#@title Install Kaolin

!pip install kaolin==0.13.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-1.13.1_cu113.html

Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-1.13.1_cu113.html


In [2]:
!pip install --upgrade numpy

  Using cached numpy-1.24.2-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [5]:
#import modules

import torch
import kaolin as kal
import numpy as np
from PIL import Image

device = torch.device('cuda:0')

In [6]:
#@title get camera params from elevation, azimuth, radius
def get_camera_from_view(elev, azim, r=3.0):
    x = r * torch.cos(elev) * torch.cos(azim)
    y = r * torch.sin(elev)
    z = r * torch.cos(elev) * torch.sin(azim)

    pos = torch.tensor([x, y, z]).unsqueeze(0)
    look_at = -pos
    direction = torch.tensor([0.0, 1.0, 0.0]).unsqueeze(0)

    camera_proj = kal.render.camera.generate_transformation_matrix(pos, look_at, direction)
    return camera_proj

In [11]:
#Mesh class

class Mesh:
  def __init__(self, obj_path):
     mesh = kal.io.obj.import_mesh(obj_path, with_normals = True)
     self.vertices = mesh.vertices.to(device)
     self.faces = mesh.faces.to(device)
     self.vertex_normals = mesh.vertex_normals.to(device)
     self.normalize_mesh()
    
  def normalize_mesh(self):
    if self.is_color:
        self.color_attributes = self.vertices[1::2]
        self.vertices = self.vertices[::2]
        
    center = self.vertices.mean(dim = 0)
    v = self.vertices - center
    scale = torch.norm(v, dim = 1).max()
    v /= scale
    self.vertices = v
     

In [12]:
#Render single image from Mesh
lights = torch.tensor([1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).to(device)
camera_projection =kal.render.camera.generate_perspective_projection(np.pi / 3).to(device)
dim = (224, 224)
def render_single_view(mesh, elev=0, azim=0, show=False, lighting=True, background=None, radius=2,
                        return_mask=False):
    # if mesh is None:
    #     mesh = self._current_mesh
    verts = mesh.vertices
    faces = mesh.faces
    n_faces = faces.shape[0]
    face_attributes = kal.ops.mesh.index_vertices_by_faces(mesh.color_attributes.unsqueeze(0).to(device), mesh.faces.to(device))


    camera_transform = get_camera_from_view(torch.tensor(elev), torch.tensor(azim), r=radius).to(device)
    face_vertices_camera, face_vertices_image, face_normals = kal.render.mesh.prepare_vertices(
        mesh.vertices.to(device), mesh.faces.to(device), camera_projection, camera_transform=camera_transform)

    image_features, soft_mask, face_idx = kal.render.mesh.dibr_rasterization(
        dim[1], dim[0], face_vertices_camera[:, :, :, -1],
        face_vertices_image, face_attributes, face_normals[:, :, -1])


    image = torch.clamp(image_features, 0.0, 1.0)
    if lighting:
        image_normals = face_normals[:, face_idx].squeeze(0)
        image_lighting = kal.render.mesh.spherical_harmonic_lighting(image_normals, lights).unsqueeze(0)
        image = image * image_lighting.repeat(1, 3, 1, 1).permute(0, 2, 3, 1).to(device)
        image = torch.clamp(image, 0.0, 1.0)

    return image

In [13]:
mesh = Mesh('alien_final.obj')
image = render_single_view(mesh, elev = np.pi / 3)
#image1 = render_single_view(mesh, elev = np.pi / 3, azim = np.pi/2)

Image.fromarray((image.squeeze(0).cpu().numpy() * 255.).astype('uint8'))
#Image.fromarray((image1.squeeze(0).cpu().numpy() * 255.).astype('uint8'))
     


AttributeError: 'Mesh' object has no attribute 'is_color'

In [8]:
!pip install imageio[ffmpeg]
!pip install imageio[pyav]

In [11]:
import imageio

mesh = Mesh('alien_final.obj')
azim_list = np.arange(0,2*np.pi, 0.01, dtype=np.float32)
video = imageio.get_writer('camera_view.mp4', mode='I', fps=60, codec='libx264', bitrate='16M')
for azim in azim_list:
    image = render_single_view(mesh, elev = np.pi / 3, azim = azim)
    video.append_data((image.squeeze(0).cpu().numpy() * 255.).astype('uint8'))
video.close()